Team github link: https://github.com/jlee588/DL4H_Team_51

Presentation link: https://www.youtube.com/watch?v=E-S_mWwDcrM

# Bring in necessary files from github


In [ ]:
!git clone https://github.com/jlee588/DL4H_Team_51

# Introduction
This is an introduction to your report, you should edit this text/markdown section to compose. In this text/markdown, you should introduce:

*   Background of the problem
  * This paper aims to solve the problem of predicting drug to drug interaction events for pharmaceutical research.
  * The importance of solving this problem is that most research has been done on whether two drugs interact with each other but not many have analyzed the consequential effects of when two drugs interact with each other. The paper also stated that "67% of elderly Americans took five or more medications in 2010-2011." This is important because with so many people taking multiple medications, it is imperative that we ensure those medications, if taken together, will still be safe to take.
  * This problem can be difficult because the most relevant and diverse features must be chosen to use in a deep neural network in order to retrieve the most informative results from the model.It is important to choose the right amount of diverse features that will all individually contribute to the final results because that is how we will get the most informative model.
  * The current state of the art methods in DDI predictions include DeepDDI, random forest, logistic regression, and K-nearest neighbor. From the paper, we will see that these methods have accuracy results of 0.8371,0.7775,0.7214, and 0.7920 respectively. They also have an area under the precision-recall curve of 0.8899,0.8349,0.7716, and 0.8400 respectively.

*   Paper explanation
  * The paper proposes a DNN-based architecture that combines a variety of drug features with deep learning to predict the DDI events. The features used in this model will be chemical substructures, targets, enzymes and pathways.
  * The innovations of this method include using semantic analysis on the descriptions of known DDIs from the DrugBank dataset to construct an interaction event dataset and then using diverse features from drugs to predict DDI events.
  * According to the paper, the results of the DDIMDL model outperformed all 4 other state of the art algorithms with an accuracy of 0.8852 and an area under the precision-recall curve of 0.9208.
  * The contributions to the research regime from this paper include a multimodal deep learning framework that outperforms other state of the art methods in regards to DDI event predictions. This can be very helpful for the pharmaceutical industry as there is now an even higher chance of drug to drug interactions being detected and the adverse effects of these drugs being taken together can be avoided.

# Scope of Reproducibility:

List hypotheses from the paper you will test and the corresponding experiments you will run.


1.   Hypothesis 1: A DNN-based architecture, specifically named DDIMDL, is superior to other state of the art algorithms in regards to predicting DDIs. To test this, our model will compare models that use the DDIMDL algorithm to predict DDIs with other state of the art algorithms such as random forest, logistic regression, gradient boosting decision tree, and K-nearest neighbor while all other aspects remain the same.
Our ablations planned are to remove one or two layers from the model and compare those results from the original model.

To reproduce the paper's results, we plan to modernize the code to work around deprecated functions and this will be shown in the commented out code throughout the file that will be replaced with the modern code below it.

For example, np.mat() was deprecated, so we used np.array() instead like below
    #     matrix = np.mat(matrix)
    #     matrix = np.array(matrix)


# Methodology

*   Environment: Google Colab Notebook with CPU runtime
*   Python: Python 3.10.2
*   Packages: Needed packages imported below



In [ ]:
# import  packages you need
import numpy as np
from google.colab import drive
from numpy.random import seed
seed(1)
import csv
import sqlite3
import time
import numpy as np
import pandas as pd
from pandas import DataFrame
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import label_binarize
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression  # Corrected import
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, Activation, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

##  Data
Data includes raw data (drug, event, interaction) tables from event.db, pre-procssed data (extraction) table from event.db, descriptive statistics (size of each table and basic info), and data processing (feature engineering).
  * Source of the data: pre-processed data using Drugbank (https://go.drugbank.com/) is available from the public github of the paper (https://github.com/YifanDengWHU/DDIMDL) as a database file (event.db). A separate NLP script (NLPProcess.py available in github) was used to extract drug interactions to form extractions table in event.db
  * Statistics: Functions to print out basic descriptive statistics are defined below.
  * Data process: Data processing functions to split the data and convert them to feature vectors are defined below.
  * Illustration: Functions for printing results of the model are defined below.

In [ ]:
'''
All raw data needed are in event.db file
event.db is also available in public github: https://github.com/jlee588/DL4H_Team_51
Each dataframe is constructed by connecting to the event.db file via a sql query as shown below
'''

raw_data_dir = '/content/DL4H_Team_51/event.db'

def load_raw_data(raw_data_dir):
  # implement this function to load raw data to dataframe/numpy array/tensor
  conn = sqlite3.connect(raw_data_dir)
  df_drug = pd.read_sql('select * from drug;', conn)
  df_event = pd.read_sql('select * from event_number;', conn)
  df_interaction = pd.read_sql('select * from event;', conn)
  return df_drug, df_event, df_interaction

df_drug, df_event, df_interaction  = load_raw_data(raw_data_dir)
conn = sqlite3.connect(raw_data_dir)


In [ ]:
# calculate statistics
'''
1.drug contains 572 kinds of drugs and their features.
2.event contains the 37264 DDIs between the 572 kinds of drugs.
3.event_number lists the kinds of DDI events and their occurence frequency.
'''
def calculate_stats(dataframes):
  # implement this function to calculate the statistics
  # it is encouraged to print out the results
  for df_name, df in dataframes.items():
        print(f"Statistics for {df_name}:")
        print("Shape:", df.shape)
        print("Info:")
        df.info()
        print("="*40)

calculate_stats({
    'Drug': df_drug,
    'Event Number': df_event,
    'Interaction': df_interaction
})

In [ ]:
'''
This function works with raw data (df_drug) table and feature_vector function
to create a feature vector with new labels
It creates and returns:
new_feature: array with combined feature vectors for drug pairs
new_label: array of labels for each drug pair interaction
event_num: number of events to be used with other functions

'''
def prepare(df_drug, feature_list, vector_size,mechanism,action,drugA,drugB):
    d_label = {}
    d_feature = {}
    # Transfrom the interaction event to number
    # Splice the features
    d_event=[]
    for i in range(len(mechanism)):
        d_event.append(mechanism[i]+" "+action[i])
    label_value = 0
    count={}
    for i in d_event:
        if i in count:
            count[i]+=1
        else:
            count[i]=1
    list1 = sorted(count.items(), key=lambda x: x[1],reverse=True)
    for i in range(len(list1)):
        d_label[list1[i][0]]=i
    vector = np.zeros((len(np.array(df_drug['name']).tolist()), 0), dtype=float)
    for i in feature_list:
        vector = np.hstack((vector, feature_vector(i, df_drug, vector_size)))
    # Transfrom the drug ID to feature vector
    for i in range(len(np.array(df_drug['name']).tolist())):
        d_feature[np.array(df_drug['name']).tolist()[i]] = vector[i]
    # Use the dictionary to obtain feature vector and label
    new_feature = []
    new_label = []
    name_to_id = {}
    for i in range(len(d_event)):
        new_feature.append(np.hstack((d_feature[drugA[i]], d_feature[drugB[i]])))
        new_label.append(d_label[d_event[i]])
    new_feature = np.array(new_feature)
    new_label = np.array(new_label)
    return (new_feature, new_label, event_num)

In [ ]:
'''
This function first defines a function to create a Jaccard similarity matrix to
be used to quantify the similarity between every pair of drugs based on shared
features.
It then extracts and compiles a list of unique features from df_drug table
Jaccard function is used to generate similarity matrices
PCA is applied to reduce the dimensionality to vector_size
'''
def feature_vector(feature_name, df, vector_size):
    # df are the 572 kinds of drugs
    # Jaccard Similarity
    # np.matrix outdated
    # def Jaccard(matrix):
    #     matrix = np.mat(matrix)
    #     numerator = matrix * matrix.T
    #     denominator = np.ones(np.shape(matrix)) * matrix.T + matrix * np.ones(np.shape(matrix.T)) - matrix * matrix.T
    #     return numerator / denominator
    def Jaccard(matrix):
        matrix = np.array(matrix)
        numerator = matrix.dot(matrix.T)
        denominator = (np.ones(matrix.shape) @ matrix.T) + (matrix @ np.ones(matrix.T.shape)) - (matrix.dot(matrix.T))
        denominator[denominator == 0] = 1
        return numerator / denominator

    all_feature = []
    drug_list = np.array(df[feature_name]).tolist()
    # Features for each drug, for example, when feature_name is target, drug_list=["P30556|P05412","P28223|P46098|……"]
    for i in drug_list:
        for each_feature in i.split('|'):
            if each_feature not in all_feature:
                all_feature.append(each_feature)  # obtain all the features
    feature_matrix = np.zeros((len(drug_list), len(all_feature)), dtype=float)
    df_feature = DataFrame(feature_matrix, columns=all_feature)  # Consrtuct feature matrices with key of dataframe
    for i in range(len(drug_list)):
        for each_feature in df[feature_name].iloc[i].split('|'):
            df_feature[each_feature].iloc[i] = 1
    sim_matrix = Jaccard(np.array(df_feature))

    sim_matrix1 = np.array(sim_matrix)
    count = 0
    pca = PCA(n_components=vector_size)  # PCA dimension
    pca.fit(sim_matrix)
    sim_matrix = pca.transform(sim_matrix)
    return sim_matrix

In [ ]:
'''
This function is used to assign labels and split the dataset for KFold cross-
validation.
It assigns a fold number 'k_num' to each test index in the index_all_classes array
'''
def get_index(label_matrix, event_num, seed, CV):
    index_all_class = np.zeros(len(label_matrix))
    for j in range(event_num):
        index = np.where(label_matrix == j)
        kf = KFold(n_splits=CV, shuffle=True, random_state=seed)
        k_num = 0
        for train_index, test_index in kf.split(range(len(index[0]))):
            index_all_class[index[0][test_index]] = k_num
            k_num += 1

    return index_all_class

##   Model
The model includes the model definitation which usually is a class, model training, and other necessary parts.
  * Model architecture: layer number/size/type, activation function, etc
  * Training objectives: loss function, optimizer, weight of each loss term, etc
  * Others: whether the model is pretrained, Monte Carlo simulation for uncertainty analysis, etc
  * The code of model should have classes of the model, functions of model training, model validation, etc.
  * If your model training is done outside of this notebook, please upload the trained model here and develop a function to load and test it.

In [ ]:
'''
The main model (DDIMDL) is defined below.
All the layers needed (Dense, Dropout, BatchNormalization, Activation) are defined with input_features and output_features
Loss function is categorical_crossentropy
Adam optimizer is used for the model
'''

def DNN():
    train_input = Input(shape=(vector_size * 2,), name='Inputlayer')
    train_in = Dense(512, activation='relu')(train_input)
    train_in = BatchNormalization()(train_in)
    train_in = Dropout(droprate)(train_in)
    train_in = Dense(256, activation='relu')(train_in)
    train_in = BatchNormalization()(train_in)
    train_in = Dropout(droprate)(train_in)
    train_in = Dense(event_num)(train_in)
    out = Activation('softmax')(train_in)
    model = Model(inputs=train_input, outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
'''
Functions are defined below to evaluate each classifier using various metrics
Results are saved as arrays and returned as outputs
We can calculate accuracy, roc_aupr, roc_auc, f1_score, precision, recall
These metrics are organized by each classifier for easy comparison
'''
def evaluate(pred_type, pred_score, y_test, event_num, set_name):
    all_eval_type = 11
    result_all = np.zeros((all_eval_type, 1), dtype=float)
    each_eval_type = 6
    result_eve = np.zeros((event_num, each_eval_type), dtype=float)
    # y_one_hot = label_binarize(y_test, np.arange(event_num))
    # pred_one_hot = label_binarize(pred_type, np.arange(event_num))
    y_one_hot = label_binarize(y_test, classes=np.arange(event_num))
    pred_one_hot = label_binarize(pred_type, classes=np.arange(event_num))

    precision, recall, th = multiclass_precision_recall_curve(y_one_hot, pred_score)

    result_all[0] = accuracy_score(y_test, pred_type)
    result_all[1] = roc_aupr_score(y_one_hot, pred_score, average='micro')
    result_all[2] = roc_aupr_score(y_one_hot, pred_score, average='macro')
    result_all[3] = roc_auc_score(y_one_hot, pred_score, average='micro')
    result_all[4] = roc_auc_score(y_one_hot, pred_score, average='macro')
    result_all[5] = f1_score(y_test, pred_type, average='micro')
    result_all[6] = f1_score(y_test, pred_type, average='macro')
    result_all[7] = precision_score(y_test, pred_type, average='micro', zero_division=0)
    result_all[8] = precision_score(y_test, pred_type, average='macro', zero_division=0)
    result_all[9] = recall_score(y_test, pred_type, average='micro')
    result_all[10] = recall_score(y_test, pred_type, average='macro')
    for i in range(event_num):
        result_eve[i, 0] = accuracy_score(y_one_hot.take([i], axis=1).ravel(), pred_one_hot.take([i], axis=1).ravel())
        result_eve[i, 1] = roc_aupr_score(y_one_hot.take([i], axis=1).ravel(), pred_one_hot.take([i], axis=1).ravel(),
                                          average=None)
        result_eve[i, 2] = roc_auc_score(y_one_hot.take([i], axis=1).ravel(), pred_one_hot.take([i], axis=1).ravel(),
                                         average=None)
        result_eve[i, 3] = f1_score(y_one_hot.take([i], axis=1).ravel(), pred_one_hot.take([i], axis=1).ravel(),
                                    average='binary')
        result_eve[i, 4] = precision_score(y_one_hot.take([i], axis=1).ravel(), pred_one_hot.take([i], axis=1).ravel(),
                                           average='binary', zero_division=0)
        result_eve[i, 5] = recall_score(y_one_hot.take([i], axis=1).ravel(), pred_one_hot.take([i], axis=1).ravel(),
                                        average='binary')
    return [result_all, result_eve]


def self_metric_calculate(y_true, pred_type):
    y_true = y_true.ravel()
    y_pred = pred_type.ravel()
    if y_true.ndim == 1:
        y_true = y_true.reshape((-1, 1))
    if y_pred.ndim == 1:
        y_pred = y_pred.reshape((-1, 1))
    y_true_c = y_true.take([0], axis=1).ravel()
    y_pred_c = y_pred.take([0], axis=1).ravel()
    TP = 0
    TN = 0
    FN = 0
    FP = 0
    for i in range(len(y_true_c)):
        if (y_true_c[i] == 1) and (y_pred_c[i] == 1):
            TP += 1
        if (y_true_c[i] == 1) and (y_pred_c[i] == 0):
            FN += 1
        if (y_true_c[i] == 0) and (y_pred_c[i] == 1):
            FP += 1
        if (y_true_c[i] == 0) and (y_pred_c[i] == 0):
            TN += 1
    print("TP=", TP, "FN=", FN, "FP=", FP, "TN=", TN)
    return (TP / (TP + FP), TP / (TP + FN))


def multiclass_precision_recall_curve(y_true, y_score):
    y_true = y_true.ravel()
    y_score = y_score.ravel()
    if y_true.ndim == 1:
        y_true = y_true.reshape((-1, 1))
    if y_score.ndim == 1:
        y_score = y_score.reshape((-1, 1))
    y_true_c = y_true.take([0], axis=1).ravel()
    y_score_c = y_score.take([0], axis=1).ravel()
    precision, recall, pr_thresholds = precision_recall_curve(y_true_c, y_score_c)
    return (precision, recall, pr_thresholds)


def roc_aupr_score(y_true, y_score, average="macro"):
    def _binary_roc_aupr_score(y_true, y_score):
        precision, recall, pr_thresholds = precision_recall_curve(y_true, y_score)
        return auc(recall, precision)

    def _average_binary_score(binary_metric, y_true, y_score, average):  # y_true= y_one_hot
        if average == "binary":
            return binary_metric(y_true, y_score)
        if average == "micro":
            y_true = y_true.ravel()
            y_score = y_score.ravel()
        if y_true.ndim == 1:
            y_true = y_true.reshape((-1, 1))
        if y_score.ndim == 1:
            y_score = y_score.reshape((-1, 1))
        n_classes = y_score.shape[1]
        score = np.zeros((n_classes,))
        for c in range(n_classes):
            y_true_c = y_true.take([c], axis=1).ravel()
            y_score_c = y_score.take([c], axis=1).ravel()
            score[c] = binary_metric(y_true_c, y_score_c)
        return np.average(score)

    return _average_binary_score(_binary_roc_aupr_score, y_true, y_score, average)

In [ ]:
'''
Training loop as well as cross validation is defined in below function
Based on what type of classifier we're using, corresponding model will be used for training
For DDIMDL, number of epochs and batch_size can be specified
'''
number_epochs = 1

def cross_validation(feature_matrix, label_matrix, clf_type, event_num, seed, CV, set_name):
    all_eval_type = 11
    result_all = np.zeros((all_eval_type, 1), dtype=float)
    each_eval_type = 6
    result_eve = np.zeros((event_num, each_eval_type), dtype=float)
    y_true = np.array([])
    y_pred = np.array([])
    y_score = np.zeros((0, event_num), dtype=float)
    index_all_class = get_index(label_matrix, event_num, seed, CV)
    matrix = []
    if type(feature_matrix) != list:
        matrix.append(feature_matrix)
        feature_matrix = matrix
    for k in range(CV):
        train_index = np.where(index_all_class != k)
        test_index = np.where(index_all_class == k)
        pred = np.zeros((len(test_index[0]), event_num), dtype=float)
        for i in range(len(feature_matrix)):
            x_train = feature_matrix[i][train_index]
            x_test = feature_matrix[i][test_index]
            y_train = label_matrix[train_index]
            # one-hot encoding
            y_train_one_hot = np.array(y_train)
            y_train_one_hot = (np.arange(y_train_one_hot.max() + 1) == y_train[:, None]).astype(dtype='float32')
            y_test = label_matrix[test_index]
            # one-hot encoding
            y_test_one_hot = np.array(y_test)
            y_test_one_hot = (np.arange(y_test_one_hot.max() + 1) == y_test[:, None]).astype(dtype='float32')
            if clf_type == 'DDIMDL':
                dnn = DNN()
                early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto')
                dnn.fit(x_train, y_train_one_hot, batch_size=128, epochs=number_epochs, validation_data=(x_test, y_test_one_hot),
                        callbacks=[early_stopping])
                pred += dnn.predict(x_test)
                continue
            elif clf_type == 'RF':
                clf = RandomForestClassifier(n_estimators=100)
            elif clf_type == 'GBDT':
                clf = GradientBoostingClassifier()
            elif clf_type == 'SVM':
                clf = SVC(probability=True)
            elif clf_type == 'FM':
                clf = GradientBoostingClassifier()
            elif clf_type == 'KNN':
                clf = KNeighborsClassifier(n_neighbors=4)
            else:
                clf = LogisticRegression(max_iter=1000)
            clf.fit(x_train, y_train)
            pred += clf.predict_proba(x_test)
        pred_score = pred / len(feature_matrix)
        pred_type = np.argmax(pred_score, axis=1)
        y_true = np.hstack((y_true, y_test))
        y_pred = np.hstack((y_pred, pred_type))
        y_score = np.row_stack((y_score, pred_score))
    result_all, result_eve = evaluate(y_pred, y_score, y_true, event_num, set_name)
    return result_all, result_eve

# Training
*   Hyperparams
  *    input layer shape: 572 * 2 = 1144
  *    batch_size: 128
  *    drop rate: 0.3
  *    optimizer: Adam
*   Computational Requirements:
  *    Hardware: Google Colab CPU runtime
  *    Avg runtime of epoch: 6 secs
  *    Total training time (DDIMDL): ~40 minutes
  *    number of epochs: 100 with EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto')



In [ ]:
'''
Using all the data processing, feature vector, model training functions defined, we can train the model
Droprate, vector_size, seed and number of CV folds can be customized
Classifiers to be used for comparison can be specified with clf_list

'''
event_num = 65
droprate = 0.3
vector_size = 572
seed = 0
CV = 5
interaction_num = 10
feature_list = ["smile","target","enzyme"] #this is default feature list
featureName="+".join(feature_list)

# clf_list = ["DDIMDL","RF","KNN"]
clf_list = ["DDIMDL"]
for feature in feature_list:
    set_name = feature + '+'
set_name = set_name[:-1]
result_all = {}
result_eve = {}
all_matrix = []
drugList=[]
for line in open("/content/DL4H_Team_51/DrugList.txt",'r'):
    drugList.append(line.split()[0])

extraction = pd.read_sql('select * from extraction;', conn)
mechanism = extraction['mechanism']
action = extraction['action']
drugA = extraction['drugA']
drugB = extraction['drugB']

for feature in feature_list:
    print(feature)
    new_feature, new_label, event_num = prepare(df_drug, [feature], vector_size, mechanism,action,drugA,drugB)
    all_matrix.append(new_feature)


for clf in clf_list:
    print(clf)
    all_result, each_result = cross_validation(all_matrix, new_label, clf, event_num, seed, CV,
                                                set_name)
    result_all[clf] = all_result
    result_eve[clf] = each_result

# with open(f'{model_save_path}/result_ablation.pkl', 'wb') as file:
#     pickle.dump(result_all, file)
# with open(f'{model_save_path}/result_eve.pkl', 'wb') as file:
#     pickle.dump(result_eve, file)


In [ ]:
#Loading trained model results
import pickle

with open('/content/DL4H_Team_51/result_all.pkl', 'rb') as file:
    result_all = pickle.load(file)
with open('/content/DL4H_Team_51/result_eve.pkl', 'rb') as file:
    result_eve = pickle.load(file)

# Ablation Study

For our ablation study, we removed the first Dropout layer from the model to see how the results will differ from the original model. To do this, we commented out the first Dropout layer and only included DDIMDL in the clf_list. The results of both the ablation study and the original model for DDIMDL is saved in the 'result_ablation' file which we will use to plot our graphs and compare the results. Our prediction was that the modified model will perform poorer than the original since dropout layers are used to prevent overfitting.




In [ ]:
with open('/content/DL4H_Team_51/result_ablation.pkl', 'rb') as file:
    result_ablation = pickle.load(file)

In [ ]:
'''
Function to generate chart with metrics for model comparison with other standard models
'''

import matplotlib.pyplot as plt

def plot_overall_evaluation_results(result_all):
    # Overall Evaluation Metrics
    overall_metrics_labels = [
        "Accuracy",
        "ROC AUPR (micro)",
        "ROC AUPR (macro)",
        "ROC AUC (micro)",
        "ROC AUC (macro)",
        "F1 Score (micro)",
        "F1 Score (macro)",
        "Precision (micro)",
        "Precision (macro)",
        "Recall (micro)",
        "Recall (macro)"
    ]

    n_classifiers = len(result_all)
    bar_width = 0.25

    indices = np.arange(len(overall_metrics_labels))
    positions = [indices + i * bar_width for i in range(n_classifiers)]

    plt.figure(figsize=(14, 8))
    for idx, (classifier_name, results) in enumerate(result_all.items()):
        overall_metrics_values = [val[0] for val in results]
        plt.bar(positions[idx], overall_metrics_values, width=bar_width, label=classifier_name)

    plt.xlabel('Score', fontsize=12)
    plt.ylabel('Metrics', fontsize=12)
    plt.title('Comparison of Overall Evaluation Metrics by Classifier', fontsize=16)
    plt.xticks([r + bar_width for r in range(len(overall_metrics_labels))], overall_metrics_labels, rotation=30)
    plt.legend()

    plt.show()

plot_overall_evaluation_results(result_ablation)

In [ ]:
def create_comparison_table(result_all):
    metrics = ["Accuracy", "ROC AUPR (micro)", "ROC AUPR (macro)", "ROC AUC (micro)",
               "ROC AUC (macro)", "F1 Score (micro)", "F1 Score (macro)",
               "Precision (micro)", "Precision (macro)", "Recall (micro)", "Recall (macro)"]

    data = {classifier_name: [result[0] for result in results] for classifier_name, results in result_all.items()}

    df = pd.DataFrame(data, index=metrics)

    return df

comparison_table = create_comparison_table(result_ablation)
print(comparison_table)

The results of this ablation study confirmed our predictions as all but one of the evaluation metrics was higher in the original model compared to the modified model.

# Evaluation

•	Accuracy: Measures the percentage of correct predictions for all classes

•	ROC AUPR (micro): The area under precision-recall curve for all classes

•	ROC AUPR (macro): The area under precision-recall curve for for each class and averaged

•	ROC AUC (micro): The area under the receiver operating characteristic curve, aggregating outcomes of all classes to evaluate the overall performance

•	ROC AUC (macro): Average area under the curve for each class, calculated separately and then averaged, giving equal weight to each class

•	F1 Score (micro): Harmonic mean of precision and recall for all classes

•	F1 Score (macro): Calculates the F1 score for each class individually and averages them

•	Precision (micro): Proportion of correct positive predictions across all classes

•	Precision (macro): Mean of precision scores for each class, calculated separately



In [ ]:
'''
Function to generate chart with metrics for model comparison with other standard models
'''

import matplotlib.pyplot as plt

def plot_overall_evaluation_results(result_all):
    # Overall Evaluation Metrics
    overall_metrics_labels = [
        "Accuracy",
        "ROC AUPR (micro)",
        "ROC AUPR (macro)",
        "ROC AUC (micro)",
        "ROC AUC (macro)",
        "F1 Score (micro)",
        "F1 Score (macro)",
        "Precision (micro)",
        "Precision (macro)",
        "Recall (micro)",
        "Recall (macro)"
    ]

    n_classifiers = len(result_all)
    bar_width = 0.25

    indices = np.arange(len(overall_metrics_labels))
    positions = [indices + i * bar_width for i in range(n_classifiers)]

    plt.figure(figsize=(14, 8))
    for idx, (classifier_name, results) in enumerate(result_all.items()):
        overall_metrics_values = [val[0] for val in results]
        plt.bar(positions[idx], overall_metrics_values, width=bar_width, label=classifier_name)

    plt.xlabel('Score', fontsize=12)
    plt.ylabel('Metrics', fontsize=12)
    plt.title('Comparison of Overall Evaluation Metrics by Classifier', fontsize=16)
    plt.xticks([r + bar_width for r in range(len(overall_metrics_labels))], overall_metrics_labels, rotation=30)
    plt.legend()

    plt.show()

plot_overall_evaluation_results(result_all)

In [ ]:
def create_comparison_table(result_all):
    metrics = ["Accuracy", "ROC AUPR (micro)", "ROC AUPR (macro)", "ROC AUC (micro)",
               "ROC AUC (macro)", "F1 Score (micro)", "F1 Score (macro)",
               "Precision (micro)", "Precision (macro)", "Recall (micro)", "Recall (macro)"]

    data = {classifier_name: [result[0] for result in results] for classifier_name, results in result_all.items()}

    df = pd.DataFrame(data, index=metrics)

    return df

comparison_table = create_comparison_table(result_all)
print(comparison_table)

# Results
*   With the notebook, we were able to achieve very similar scores for all metrics for the DDIMDL model
*   As hypothesized, DDIMDL model performed better than other standard classifiers for all metrics. Biggest differences can be seen from F1 Score (macro) and Recall (macro)
*   Ablation study was performed with one layer removed and it performed worse than the original setup of the DDIMDL model. This was also stated by the paper and proven with results.



# Discussion
*   The paper is fully reproducible in Google Colab Notebook environment
*   What was easy
  *    Working with clearly defined sections/functions for fitting into Colab Notebook
  *    Not too complex code for feature engineering, training, cross-validation
  *    Easy to test other types of models for comparison using built in functions
*    What was difficult
  *    Figuring out what packages/functions are deprecated with Colab packages
  *    Modernizing inputs and outputs of functions that require different syntax
  *    Rewriting functions to have them work with one notebook instead of working with several different script files
*   Suggestions to the author
  *    Use modern packages and rewrite couple functions to make them work with modern packages
  *    Clearer instructions for initializing the model with user inputs



# References

1.   @article{deng2020multimodal,
title={A multimodal deep learning framework for predicting drug-drug interaction events},
author={Deng, Yifan and Xu, Xinran and Qiu, Yang and Xia, Jingbo and Zhang, Wen and Liu, Shichao},
journal={Bioinformatics}
}

2. https://github.com/YifanDengWHU/DDIMDL
